In [1]:
import numpy as np
import pandas as pd
import plotly_express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from blackscholes import BlackScholesCall, BlackScholesPut
from scipy.stats import gmean, norm
from scipy.optimize import curve_fit, minimize, differential_evolution, NonlinearConstraint
from pyra.date_utils import hourly_index
import sys
from getpass import getpass
from pandas.tseries.offsets import MonthEnd

### EMTDB Connection

In [2]:
sys.path.append(r"K:\Valuation\_Analysts\Hemanth\Python Notebooks\Miscellaneous\Python Analyst Engine 2.0")

from util import EmtdbConnection, date_hour_to_peak_block, hourly_index
from emtdb_api import pull_lmp_data, pull_system_vols, pull_fwd_market_price, pull_projection_curves

user = 'HXH07BP'
pw = getpass('Enter EMTDB pass:')

emtdb = EmtdbConnection(user, pw)

connecting to EMTDB...
connected.


### Helper functions

In [3]:
def euro_option_price(s_0, k, T, r, sigma, div=0, div_yield=0, call=1):
  d1 = (np.log((s_0 - div) / k) + ((r - div_yield) + (sigma ** 2) / 2) * T) / (sigma * np.sqrt(T))
  d2 = d1 - sigma * np.sqrt(T)

  if call:
    return (s_0 - div) * np.exp(-div_yield * T) * norm.cdf(d1) - k * np.exp(-r * T) * norm.cdf(d2)
  else: # put
    return k * np.exp(-r * T) * norm.cdf(-d2) - (s_0 - div) * np.exp(-div_yield * T) * norm.cdf(-d1)
  
def euro_gap_option_price(s_0, k1, k2, T, r, sigma, div=0, call=1):
  d1 = (np.log((s_0 - div) / k2) + (r + (sigma ** 2) / 2) * T) / (sigma * np.sqrt(T))
  d2 = d1 - sigma * np.sqrt(T)

  if call:
    return (s_0 - div) * norm.cdf(d1) - k1 * np.exp(-r * T) * norm.cdf(d2)
  else: # put
    return k1 * np.exp(-r * T) * norm.cdf(-d2) - (s_0 - div) * norm.cdf(-d1)
  
def delta(s_0, k, T, r, sigma, div=0, call=1):
  d1 = (np.log((s_0 - div) / k) + (r + (sigma ** 2) / 2) * T) / (sigma * np.sqrt(T))

  if call:
    return norm.cdf(d1)
  else: # put
    return norm.cdf(d1) - 1

## Simulating GBM

Note that the code below simulates GBM in two different ways:
1. Step-wise: Where the successive sample depends on the previous sample. Calculations of volatility and correlation are based on the difference between successive samples.

2. T-0: Where every sample is based on the initial (t = 0) value. Samples that are further out in time have higher variance, and hence this leads to a more spiky path than step-wise GBM. Calculations of volatility and correlation are based on the difference between a sample and the initial value.

Note that method 1 is what should be used for valuing path-dependent derivatives.

### GBM function

In [ ]:
def gbm(init_load_price: np.array, mu_load_price: np.array, sigma_load_price: np.array, load_price_corr: float, time_range: int, num_iterations: int, step_wise_GBM: bool):
    
    """
    Generates load and price paths at an hourly frequency based on a correlated geometric brownian motion

    Args:
        init_load_price: (2, ) array with initial load and price
        mu_load_price: (2, ) array with drift of load and price
        sigma_load_price: (2, ) array with volatility of load and price
        load_price_corr: Load-price correlation
        time_range: number of hours for which to simulate loads and prices
        step_wise_GBM: True if step_wise and False otherwise  

    Returns: 
        time_arr (time_range, ) array with times
        loads_prices_arr (2, time_range, num_iterations) array with the loads and prices simulated over the specified {time_range} {num_iterations} times
    """

    load_price_corr_mat = np.array(
    [
        [1, load_price_corr],
        [load_price_corr, 1]
    ]
)
    
    loads_prices_arr = np.zeros((2, time_range, num_iterations)) # first row - load, second row - prices

    time_arr = np.arange(0, time_range) / 8760 # in years

    loads_prices_arr[:, 0, :] = np.column_stack([init_load_price] * num_iterations)

    delta_t = time_arr[1] - time_arr[0] # time between successive samples in years

    iid_std_normals = np.random.randn(2, time_range, num_iterations) # independent standard normals

    corr_std_normals = (np.linalg.cholesky(load_price_corr_mat) @ iid_std_normals.transpose(1, 0, 2)).transpose(1, 0, 2) # correlated standard normals

    if step_wise_GBM:
        
        for t in range(time_range - 1):

            drift_term_load_price = mu_load_price.reshape(-1, 1) * delta_t * loads_prices_arr[:, t, :]
            noise_term_load_price = corr_std_normals[:, t + 1, :] * sigma_load_price.reshape(-1, 1) * np.sqrt(delta_t) * loads_prices_arr[:, t, :]  
            loads_prices_arr[:, t + 1, :] = loads_prices_arr[:, t, :] + drift_term_load_price + noise_term_load_price

        # These can be printed for a single iteration
        # print('Calculated load volatility', np.std(np.diff(np.log(loads_prices_arr[0, :])) / np.sqrt(delta_t))) # standard deviation of log return of load 
        # print('Calculated price volatility', np.std(np.diff(np.log(loads_prices_arr[1, :])) / np.sqrt(delta_t))) # standard deviation of log return of price
        # print('Calculated correlation of log return', np.corrcoef(np.diff(loads_prices_arr[0, :]), np.diff(np.log(loads_prices_arr[1, :])))[0, 1])
        
    else: # Based on starting point at time = 0
        
        drift_term = np.exp(((mu_load_price.reshape(-1, 1) - (sigma_load_price.reshape(-1, 1) ** 2) / 2) * time_arr))
        drift_term_broadcasted = np.stack([drift_term] * num_iterations, axis=0).transpose(1, 2, 0)

        noise_term_broadcasted = np.exp(corr_std_normals * (sigma_load_price.reshape(-1, 1) * np.sqrt(time_arr))[:, :, np.newaxis])
        
        loads_prices_arr = loads_prices_arr[:, 0, :][:, np.newaxis, :] * drift_term_broadcasted * noise_term_broadcasted

        # These can be printed for a single iteration
        # print('Calculated load volatility', np.std(np.log(loads_prices_arr[0, 1:] / loads_prices_arr[0, 0]) / np.sqrt(time_arr[1: ])))
        # print('Calculated price volatility', np.std(np.log(loads_prices_arr[1, 1:] / loads_prices_arr[1, 0]) / np.sqrt(time_arr[1: ])))
        # print('Calculated correlation of log', np.corrcoef(np.log(loads_prices_arr[0, :] / loads_prices_arr[0, 0]), np.log(loads_prices_arr[1, :] / loads_prices_arr[1, 0]))[0, 1])

    return time_arr, loads_prices_arr

### Valuation of various vanilla and exotic derivatives

Analyst inputs (in the cell below):
1. Initial load and price 
2. Load-price correlation
3. Drift and volatlity of load and price
4. Time range of simulation
5. Type of GBM to simulate (step-wise or T-0) 
6. Number of iterations

In [ ]:
init_load_price = np.array([50, 50]) 
load_price_corr = 0.8 # load-price correlation

mu_load_price = np.array([0, 0])  # expected rate of return per year
sigma_load_price = np.array([0.25, 0.4])  # volatility of return per year

k = 50 # Strike price

time_range = 365 * 24 # Number of hours (to simulate loads and prices)

step_wise_GBM = True # Type of GBM to simulate

num_iterations = 10000

In [ ]:
time_arr, loads_prices_arr = gbm(init_load_price, mu_load_price, sigma_load_price, load_price_corr, time_range, num_iterations, step_wise_GBM)

# Valuing different options based on {num_iteration} price paths

init_load = init_load_price[0]
init_price = init_load_price[1] # Initial price of underlying

mu_load = mu_load_price[0]
mu_price = mu_load_price[1]

sigma_load = sigma_load_price[0]
sigma_price = sigma_load_price[1]

T = time_range / 8760 # Time to expiry in years 
r = 0 # risk-free rate
sigma = sigma_load_price[1] # Volatility of underlying

if step_wise_GBM:
    print(f'Empirical load volatility {np.average(np.std(np.diff(np.log(loads_prices_arr[0, :, :]), axis=0), axis=0) / np.sqrt(1 / 8760)):.2f}')
    print(f'Empirical price volatility {np.average(np.std(np.diff(np.log(loads_prices_arr[1, :, :]), axis=0), axis=0) / np.sqrt(1 / 8760)):.2f}')
    print(f'Empirical load-price correlation (using the log returns of load and price) {np.array([np.corrcoef(np.diff(np.log(loads_prices_arr[0, :, i])), np.diff(np.log(loads_prices_arr[1, :, i])))[0, 1] for i in range(num_iterations)]).mean():.2f}\n')
else:
    print(f'Empirical load volatility {np.std((np.log(loads_prices_arr[0, :, :]))[1:, :] / np.sqrt(time_arr)[1: ].reshape(-1, 1), axis=1).mean():.2f}')
    print(f'Empirical price volatility {np.std((np.log(loads_prices_arr[1, :, :]))[1:, :] / np.sqrt(time_arr)[1: ].reshape(-1, 1), axis=1).mean():.2f}')
    print(f'Empirical load-price correlation (using the log of load and price) {np.array([np.corrcoef(np.log(loads_prices_arr[0, :, i])[1:],np.log(loads_prices_arr[1, :, i])[1:])[0, 1] for i in range(time_range)]).mean():.2f}\n')

# Discounting is taken into account in all of the valuations below

print('Verifying load-price covariance formula (what the FR desk calls LFA):')
print(f'Empirical value of covariance {np.exp(-mu_price * T) * np.cov(loads_prices_arr[0, time_range - 1, :], loads_prices_arr[1, time_range - 1, :])[0, 1]:.2f}')
print(f'Theoretical value of covariance {init_load * init_price * np.exp((mu_load) * T) * (np.exp(sigma_load * sigma_price * load_price_corr * T) - 1):.2f}\n')

print('Verifying value of path-independent European options: \n')
print(f'Empirical value of European call {np.exp(-mu_price * T) * np.mean(np.maximum(loads_prices_arr[1, time_range - 1, :] - k, 0)):.2f}')
print(f'Theoretical value of European call {BlackScholesCall(init_price, k, 1, r=r, sigma=sigma, q=0).price():.2f}\n')
print(f'Empirical value of European put {np.exp(-mu_price * T) * np.mean(np.maximum(k - loads_prices_arr[1, time_range - 1, :], 0)):.2f}')
print(f'Theoretical value of European put {BlackScholesPut(init_price, k, 1, r=r, sigma=sigma, q=0).price():.2f}\n')

# Gap option
k1 = 100
k2 = 150

print(f'Empirical value of European gap call {np.exp(-mu_price * T) * np.mean(np.where(loads_prices_arr[1, time_range - 1, :] > k2, loads_prices_arr[1, time_range - 1, :] - k1, 0)):.2f}')
print(f'Theoretical value of European gap call {euro_gap_option_price(init_price, k1, k2, 1, r=r, sigma=sigma):.2f}\n')

print('Calculating value of path-dependent exotic options:\n')
print(f'Empirical value of Asian call {(np.exp(-mu_price * T) * np.mean(np.maximum(loads_prices_arr[1, :, :].mean(axis=0) - k, 0))):.2f}')
print(f'Empirical value of lookback call with fixed strike {np.exp(-mu_price * T) * np.mean(np.maximum(np.max(loads_prices_arr[1, :, :], axis=0) - k, 0)):.2f}')
print(f'Empirical value of lookback call with floating strike {np.exp(-mu_price * T) * np.mean(loads_prices_arr[1, time_range - 1, :] - np.min(loads_prices_arr[1, :, :], axis=0)):.2f}')

knock_in = 500

print(f'Empirical value of up and in barrier option {np.exp(-mu_price * T) * np.maximum((loads_prices_arr[1, :, :] * (loads_prices_arr[1, :, :] > knock_in).astype(int).max(axis=0))[time_range - 1, :] - k, 0).mean():.2f}')

fig = go.Figure()

for iter in range(min(num_iterations, 3)): # Capping the number of load and price paths displayed at 3
    fig.add_trace(go.Scatter(name=f'Load Path {iter + 1}', x=time_arr, y=loads_prices_arr[0, :, iter]))
    fig.add_trace(go.Scatter(name=f'Price Path {iter + 1}', x=time_arr, y=loads_prices_arr[1, :, iter]))

fig.update_layout(
    title='Load and price paths',
    xaxis_title='Time (years)',
    yaxis_title='Price'
)

fig.show()

Theoretical value of Asian call - continuous and discrete case

In [ ]:
# Calculating the first two moments of the average and fitting that to a lognormal distribution

average_type = 'discrete'

if average_type == 'continuous': # This runs into an error if mu_price is 0, so set it to something like 1e-10 if needed
    M_1 = init_price * (np.exp(mu_price * T) - 1) / (mu_price * T)
    M_2_first_term = (2 * init_price** 2 * np.exp((2 * mu_price + sigma_price**2) * T)) / ((mu_price + sigma_price**2) * (2 * mu_price + sigma_price**2) * T**2)
    M_2_second_term = 2 * init_price** 2 * (1 / (2 * mu_price + sigma**2) - np.exp(mu_price * T) / (mu_price + sigma_price** 2)) / (mu_price * T**2)

elif average_type == 'discrete':
    futures_arr = init_price * np.exp(mu_price * time_arr)
    M_1 = np.sum(futures_arr) / time_range
    M_2_first_term = np.sum(futures_arr**2 * np.exp(sigma_price**2 * time_arr)) / time_range**2
    # Create meshgrids for i and j indices
    i_idx, j_idx = np.meshgrid(np.arange(time_range), np.arange(time_range), indexing='ij')
    # Create mask where i < j
    mask = i_idx < j_idx
    # Vectorized calculation
    M_2_second_term = 2 * np.sum(futures_arr[i_idx] * futures_arr[j_idx] * np.exp(sigma_price**2 * time_arr[i_idx]) * mask) / time_range**2

M_2 = M_2_first_term + M_2_second_term

asian_vol = np.sqrt(np.log(M_2 / M_1** 2) / T)

euro_option_price(
    s_0=M_1,
    k=k,
    T=T,
    r=mu_price,
    sigma=asian_vol,
    div_yield=mu_price,
    call=1
)

Distribution of geometric mean of prices - should be exactly lognormal while arithmetic mean should be approximately lognormal

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Histogram(
        x=gmean(loads_prices_arr[1, :, :], axis=0),
        name='Geometric mean',
        opacity=0.75
    )
)

fig.add_trace(
    go.Histogram(
        x=np.mean(loads_prices_arr[1, :, :], axis=0),
        name='Arithmetic mean',
        opacity=0.75
    )
)

fig.update_layout(
    barmode='overlay',
    xaxis=dict(
        title='Price'
    ),
    yaxis=dict(
        title='Observations'
    )
    )

fig.show()

### Proving that the roundabout approach to calculate vol used by FR matches what was used to simulate it

In [ ]:
simulated_load = pd.DataFrame(loads_prices_arr[0, :, np.random.randint(num_iterations)]).rename(
    columns={
        0: 'Load'
    }
).set_index(
    hourly_index(
        '2014-01-01', '2014-12-31'
    )
).reset_index().rename(
    columns={
        'date': 'Date',
        'hour': 'Hour'
    }
).assign(
    # Log_load=lambda DF: np.log(DF.Load),
    Month_start=lambda DF: DF.Date.dt.to_period('M').dt.to_timestamp(),
    return_Load=lambda DF: DF.Load / DF.Load.shift(1)
)

simulated_load

In [ ]:
(simulated_load.pivot_table(
    index='Month_start',
    values='Load', # For T-0 GBM
    # values='return_Load',   # For step-wise GBM
    aggfunc='std'
) / simulated_load.pivot_table(
    index='Month_start',
    values='Load', # For T-0 GBM
    # values='return_Load', # For step-wise GBM
    aggfunc='mean'
)).rename(
    columns={
        'Load': 'Coeff_variation' # For T-0 GBM
        # 'return_Load': 'Coeff_variation' # For step-wise GBM
    }
).assign(
    Time_to_middle=lambda DF: ((DF.index + pd.Timedelta(days=15)) - (DF.index)[0]) / (365 * pd.Timedelta(days=1)),
    Vol=lambda DF: np.sqrt(np.log(1 + (DF.Coeff_variation ** 2)) / DF.Time_to_middle), # For T-0 GBM
    # Vol=lambda DF: np.sqrt(np.log(1 + (DF.Coeff_variation ** 2)) / (1 / 8760)), # For step-wise GBM
).pipe(
    px.line,
    y='Vol',
    title=f'Load vol calculated using alternate approach for simulated loads with a vol of {sigma_load}'
)

### Checking for real-world data

In practice, I think real-world data follows step-wise GBM (and not T-0 GBM). Hence I have made the comparison based on the math that applies to step-wise. I try using both hourly and daily (hourly aggregated to the daily level) data. Both approaches show that the alternate method works well.

In [ ]:
df_lmp = pull_lmp_data(
    emtdb=emtdb,
    # pnode_id='51288',
    pnode_id='4000',
    da_or_rt='DA',
    start_dt=(pd.Timestamp.today().date() - MonthEnd(1)) - pd.offsets.MonthBegin(120),
    end_dt=pd.Timestamp.today().date() - MonthEnd(1),
    price_data_type='PRICE'
).reset_index().assign(
    peak_type=lambda DF: DF.apply(lambda row: date_hour_to_peak_block(row['Date'], row['Hour'], 'PJM'), axis=1)
).pivot_table(
    index='Date',
    columns='peak_type', # Getting the 5x16, 2x16 and 7x8 average daily prices
    values='Price',
    aggfunc='mean'
).reset_index().drop(
    columns=['2x16', '7x8']
).rename(
    columns={
        '5x16': 'Price' # Dropping the OFF peak hours and retaining only the ON peak hours
    }
).assign(
    Month_start=lambda DF: DF.Date.dt.to_period('M').dt.to_timestamp(),
    Time_delta=lambda DF: (DF.Date - DF.Date.shift(1)) /  pd.Timedelta(days=365.25), # In days
    return_Price=lambda DF: DF.Price / DF.Price.shift(1),
    log_return_Price=lambda DF: np.log(DF.return_Price),
    return_Price_by_sqrt_time=lambda DF: DF.return_Price / np.sqrt(DF.Time_delta),
    log_return_Price_by_sqrt_time=lambda DF: DF.log_return_Price / np.sqrt(DF.Time_delta)
)

df_lmp

In [ ]:
daily_vol = df_lmp.pivot_table(
    index='Month_start',
    values='log_return_Price_by_sqrt_time',
    aggfunc='std'
)

daily_vol

In [ ]:
alternate_vol = (df_lmp.pivot_table(
    index='Month_start',
    values='return_Price_by_sqrt_time',
    aggfunc='std'
).rename(columns={
    'return_Price_by_sqrt_time': 'return_Price'
}) / df_lmp.pivot_table(
    index='Month_start',
    values='return_Price',
    aggfunc='mean'
)).rename(
    columns={
        'return_Price': 'Coeff_variation'
    }
).assign(
    # alternate_Vol=lambda DF: np.sqrt(np.log(1 + (DF.Coeff_variation ** 2)) / (1 / 8760))
    # alternate_Vol=lambda DF: np.sqrt(np.log(1 + (DF.Coeff_variation ** 2)) / (1 / 365))
    alternate_Vol=lambda DF: DF.Coeff_variation #/ np.sqrt(1 / 365) # This approximation also seems to work well - document this
)

alternate_vol

In [ ]:
alternate_vol.merge(
    daily_vol,
    left_index=True,
    right_index=True,
).reset_index().pipe(
    px.line,
    x='Month_start',
    # y=['alternate_Vol', 'hourly_Vol'],
    y=['alternate_Vol', 'log_return_Price_by_sqrt_time'],
    markers=True
)

Now that we have calculated the realized ON daily vol, we can see how this compares to what the options market implied right before the month started

In [ ]:
pull_system_vols(
    emtdb=emtdb,
    # cd='PJM-ON',
    cd='NEPP-ON',
    eval_dt='2015-12-01',
    first_contract_month='201601',
    last_contract_month='202512'
).reset_index().sort_values(
    by=['EFFECTIVE_DATE', 'CONTRACT_MONTH']
).drop_duplicates(
    'CONTRACT_MONTH',
    keep='last' # Essentially I am only retaining the vol implied by the options market right before the month starts
).assign(
    Month_start=lambda DF: pd.to_datetime(DF.CONTRACT_MONTH, format='%Y%m')
).sort_values('Month_start').merge(
    daily_vol,
    left_on='Month_start',
    right_index=True
).pipe(
    px.line,
    x='Month_start',
    y=['log_return_Price_by_sqrt_time', 'DAILY_VOLATILITY'], # This compares the realized daily vol to what the options market implied right before the contract month started. 
    markers=True
)

### Comparing with Jordan's code as a check

In [ ]:
from pvm import _get_cash_vol

In [ ]:
_get_cash_vol(
    emtdb,
    iso='ISONE',
    pnode_id='4000',
    start_dt=(pd.Timestamp.today().date() - MonthEnd(1)) - pd.offsets.MonthBegin(120),
    end_dt=pd.Timestamp.today().date() - MonthEnd(1),
    zero_mean=False
).reset_index().rename(
    columns={
        'index': 'Month_end'
    }
).assign(
    Month_start=lambda DF: DF.Month_end.dt.to_period('M').dt.to_timestamp()
)[['Month_start', '5x16']].merge(
    daily_vol,
    left_on='Month_start',
    right_index=True
).pipe(
    px.line,
    x='Month_start',
    y=['5x16', 'log_return_Price_by_sqrt_time'], # 5x16 is Jordan's vol and the other one is mine, as you can see they are quite similar
    markers=True
)

### Checking if data follows GBM

In [ ]:
df_lmp = pull_lmp_data(
    emtdb=emtdb,
    pnode_id='51288',
    da_or_rt='DA',
    start_dt=(pd.Timestamp.today().date() - MonthEnd(1)) - pd.offsets.MonthBegin(1),
    end_dt=pd.Timestamp.today().date() - MonthEnd(1),
    price_data_type='PRICE'
).reset_index().assign(
    log_Price=lambda DF: np.log(DF.Price),
    log_return_Price=lambda DF: DF.log_Price.diff(),
    State=lambda DF: pd.cut(
        DF.log_return_Price,
        bins=[-np.inf, -0.1, 0, 0.1, np.inf],
        labels=[1, 2, 3, 4]
    )
)

df_lmp

In [ ]:
df_lmp.pipe(
    px.histogram,
    x='log_return_Price',
    nbins=100
)

In [ ]:
df_lmp.dropna().sort_values('State').pipe(
    px.histogram,
    x='log_return_Price',
    facet_col='State',
    nbins=100
)

In [ ]:
df_lmp.rename(
    columns={
        'State': 'State_Before'
    }
).assign(
    State_After=lambda DF: DF.State_Before.shift(-1)
).pivot_table(
    index='State_Before',
    columns='State_After',
    values='Price',
    aggfunc='count'
).pipe(
    px.imshow
).update_yaxes(
    tickvals=[1, 2, 3, 4]
)

## Correlation matrices testing

Inputs

In [ ]:
num_days = 31
num_blocks = 6
day_day_corr = 0.93
block_block_corr = 0.8
load_price_corr = 0.75

Setting up load-price correlation matrix

In [ ]:
load_price_corr_mat = np.array(
    [
        [1, load_price_corr],
        [load_price_corr, 1]
    ]
)

Setting up block-to-block correlation matrix

In [ ]:

block_corr_mat = np.zeros((num_blocks, num_blocks))
np.fill_diagonal(block_corr_mat, 1)
block_corr_mat[~np.eye(block_corr_mat.shape[0], dtype=bool)] = block_block_corr

# block_corr_mat

Setting up day-to-day correlation matrix

In [ ]:
day_corr_mat = np.zeros((num_days, num_days))
np.fill_diagonal(day_corr_mat, 1)
day_corr_mat[~np.eye(day_corr_mat.shape[0], dtype=bool)] = day_day_corr

# day_corr_mat

Calculating Kronecker product of block-to-block and day-to-day matrices

In [ ]:
combined_block_day_corr_mat = np.kron(block_corr_mat, day_corr_mat)

px.imshow(combined_block_day_corr_mat, width=600, height=600)

Creating random price and load matrices

In [ ]:
random_prices = np.random.randn(num_days, num_blocks)
random_loads = np.random.randn(num_days, num_blocks)

Calculating correlated random price and load matrices

In [ ]:
correlated_prices = np.linalg.cholesky(day_corr_mat) @ random_prices @ np.linalg.cholesky(block_corr_mat)
correlated_loads = np.linalg.cholesky(day_corr_mat) @ random_loads @ np.linalg.cholesky(block_corr_mat)

Calculating 3D corr price and load matrix

In [ ]:
corr_mat_3D = np.zeros((num_days, 2, num_blocks))

for day in range(num_days):
    corr_mat_3D[day, :, :] = np.linalg.cholesky(load_price_corr_mat) @ np.stack((correlated_loads[day, :], correlated_prices[day, :]))

corr_mat_3D.shape

## LFA Visualized

In [ ]:
fwd_price = 100
t = 0.5
price_vol = 1.1

load_vol_range = np.linspace(0, 1, 100)
corr_range = np.linspace(-1, 1, 100)

load_vols, corrs = np.meshgrid(load_vol_range, corr_range)

lfa = fwd_price * (np.exp((corrs * load_vols * price_vol * t)) - 1)

fig = go.Figure(data=[go.Surface(z=lfa, x=load_vol_range, y=corr_range)])

fig.update_layout(
    title='LFA Visualized',
    scene=dict(
        xaxis_title='Load Volatility',
        yaxis_title='Correlation',
        zaxis_title='LFA'
    )
)

fig.show()

## Modeling volume as a function of price

In [4]:
df_notional_price = pd.read_excel( # Using BGE RES as an example
    'K:\Valuation\_Analysts\Hemanth\Miscellaneous\Variable volume swap research\Variable volume swap.xlsx',
    usecols=[1, 2, 3, 4],
    skiprows=1
)

df_notional_price

<>:2: SyntaxWarning: invalid escape sequence '\V'
<>:2: SyntaxWarning: invalid escape sequence '\V'
C:\Users\HXH07BP\AppData\Local\Temp\ipykernel_27592\3034128501.py:2: SyntaxWarning: invalid escape sequence '\V'
  'K:\Valuation\_Analysts\Hemanth\Miscellaneous\Variable volume swap research\Variable volume swap.xlsx',


,Date,Hour,Price,Notional
0,2014-01-01,1,39.609701,1748.293776
1,2014-01-01,2,35.274631,1681.401390
2,2014-01-01,3,34.715221,1648.163096
3,2014-01-01,4,34.715479,1642.657074
4,2014-01-01,5,35.334245,1668.956095
...,...,...,...,...
102259,2025-08-31,20,55.115302,2346.032325
102260,2025-08-31,21,38.012929,2247.661898
102261,2025-08-31,22,27.923223,2115.144191
102262,2025-08-31,23,26.148632,1905.028199


Trying to fit a call spread to all of the data numerically

In [5]:
def call_spread(S_T, N_L, N_H, K_L, K_H):
    lev = (N_H - N_L) / (K_H - K_L)
    term_1 = np.maximum(S_T - K_L, 0)
    term_2 = np.maximum(S_T - K_H, 0)
    return N_L + lev * (term_1 - term_2)

def least_squares(params, S_T_data, N_observed):
    N_L, N_H, K_L, K_H = params
    N_predicted = call_spread(S_T_data, N_L, N_H, K_L, K_H)
    return np.sum((N_observed - N_predicted) ** 2)

# def constraint_func(x): # Tried this for differential evolution
#     """Return array where all values should be >= 0"""
#     return [
#         x[3] - x[2] - 1,  # K_H > K_L + 1
#         x[1] - x[0]       # N_H > N_L
#     ]

x0 = [
    df_notional_price.Notional.quantile(0.1),   # N_L
    df_notional_price.Notional.quantile(0.9),   # N_H
    df_notional_price.Price.quantile(0.1),      # K_L
    df_notional_price.Price.quantile(0.9)       # K_H  
] # initial guess

constraints = [
    {'type': 'ineq', 'fun': lambda x: x[3] - x[2]},
    {'type': 'ineq', 'fun': lambda x: x[1] - x[0]},
    # {'type': 'ineq', 'fun': lambda x: x[0]},
    # {'type': 'ineq', 'fun': lambda x: x[1]},
    # {'type': 'ineq', 'fun': lambda x: x[2]},
    # {'type': 'ineq', 'fun': lambda x: x[3]},
    # {'type': 'ineq', 'fun': lambda x: x[0] - df_notional_price.Notional.min()},
    # {'type': 'ineq', 'fun': lambda x: df_notional_price.Notional.max() - x[1]},
    # {'type': 'ineq', 'fun': lambda x: x[2] - df_notional_price.Price.min()},
    # {'type': 'ineq', 'fun': lambda x: df_notional_price.Price.max() - x[3]},
    ]

bounds = [
    (df_notional_price.Notional.min(), df_notional_price.Notional.max()),  # N_L
    (df_notional_price.Notional.min(), df_notional_price.Notional.max()),  # N_H 
    (df_notional_price.Price.min(), df_notional_price.Price.max()),  # K_L
    (df_notional_price.Price.min(), df_notional_price.Price.max())   # K_H
]

# curve_fit(call_spread, df_notional_price.Price, df_notional_price.Notional, p0=p0)[0] # Did not work properly - so tried scipy.optimize.minimize instead 

solution = minimize(least_squares, x0, args=(df_notional_price.Price, df_notional_price.Notional), bounds=bounds,
                  method='SLSQP', constraints=constraints).x

N_L, N_H, K_L, K_H = solution[0], solution[1], solution[2], solution[3]

print(f'Optimal parameters that minimize least squares:\nN_L {N_L:.2f}\nN_H {N_H:.2f}\nK_L {K_L:.2f}\nK_H {K_H:.2f}')

S_T = np.linspace(0, 500, 100)
y = call_spread(S_T, N_L, N_H, K_L, K_H)

# fig = go.Figure()

# fig.add_scatter(
#     x=df_notional_price['Price'],
#     y=df_notional_price['Notional'],
#     mode='markers',
#     name='Actual Data',
#     # marker=dict(size=5, opacity=0.6)
# )

# fig.add_scatter(
#     x=S_T,
#     y=y,
#     mode='markers',
#     name='Call spread best fit',
#     marker=dict(size=8, color='red')
# )

# fig.update_layout(xaxis_title='Price', yaxis_title='Load')
# fig.show()

# fig.write_image('plot.png', width=1200, height=400)

Optimal parameters that minimize least squares:
N_L 994.50
N_H 2144.68
K_L 20.12
K_H 72.23


In [6]:
# Tried differential evolution - was taking very long, so killed it

# differential_evolution(
#     least_squares,
#     bounds,
#     args=(df_notional_price.Price.values, df_notional_price.Notional.values),
#     constraints=NonlinearConstraint(constraint_func, 0, np.inf),
#     seed=42,
#     maxiter=100,      # Reduce from default (1000)
#     popsize=10,       # Reduce from default (15)
#     workers=-1,       # Use all cores
#     disp=True,
#     polish=True,      # Local optimization at end
#     atol=1,           # Don't need extreme precision
#     tol=0.01
# )

In order to capture the seasonality, I am trying to fit a call spread to the data monthly 

In [7]:
# Setting up DF to store results of optimization

df_results = pd.DataFrame(columns = ['Month', 'N_L', 'N_H', 'K_L', 'K_H'])

df_results['Month'] = [i + 1 for i in range(12)]

df_results

,Month,N_L,N_H,K_L,K_H
0,1,NaN,NaN,NaN,NaN
1,2,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN
3,4,NaN,NaN,NaN,NaN
4,5,NaN,NaN,NaN,NaN
5,6,NaN,NaN,NaN,NaN
6,7,NaN,NaN,NaN,NaN
7,8,NaN,NaN,NaN,NaN
8,9,NaN,NaN,NaN,NaN
9,10,NaN,NaN,NaN,NaN


In [8]:
# This creates a grid

# fig = make_subplots(
#     rows=4, cols=3,
#     subplot_titles=[f'Month {i+1}' for i in range(12)]
# )

for month in range(1, 13):

    # For grid
    row = (month - 1) // 3 + 1  # Calculate row (1-indexed)
    col = (month - 1) % 3 + 1   # Calculate column (1-indexed)

    df_month = df_notional_price.assign(
        Month=lambda DF: DF.Date.dt.month
    ).loc[
        lambda DF: DF.Month == month 
    ]

    x0 = [
    df_month.Notional.quantile(0.3),   # N_L
    df_month.Notional.quantile(0.7),   # N_H
    df_month.Price.quantile(0.3),      # K_L
    df_month.Price.quantile(0.7)       # K_H  
    ] # initial guess

    bounds = [
    (df_month.Notional.min(), df_month.Notional.max()),  # N_L
    (df_month.Notional.min(), df_month.Notional.max()),  # N_H 
    (df_month.Price.min(), df_month.Price.max()),  # K_L
    (df_month.Price.min(), df_month.Price.max())   # K_H
    ]

    solution = minimize(least_squares, x0, args=(df_notional_price.Price, df_notional_price.Notional), bounds=bounds,
                  method='SLSQP', constraints=constraints).x

    N_L, N_H, K_L, K_H = solution[0], solution[1], solution[2], solution[3]

    # Updating the DataFrame

    df_results.loc[df_results.Month == month, ['N_L', 'N_H', 'K_L', 'K_H']] = [N_L, N_H, K_L, K_H]

    # print(f'Optimal parameters for month {month} that minimize least squares:\nN_L {N_L:.2f}\nN_H {N_H:.2f}\nK_L {K_L:.2f}\nK_H {K_H:.2f}')

    S_T = np.linspace(df_month.Price.min(), df_month.Price.max(), 100)
    y = call_spread(S_T, N_L, N_H, K_L, K_H)

    # These creates one figure below the other
    # fig = go.Figure()

    # fig.add_scatter(
    #     x=df_month['Price'],
    #     y=df_month['Notional'],
    #     mode='markers',
    #     name='Actual Data',
    #     # marker=dict(size=5, opacity=0.6)
    # )

    # fig.add_scatter(
    #     x=S_T,
    #     y=y,
    #     mode='markers',
    #     name='Call spread best fit',
    #     marker=dict(size=8, color='red')
    # )

    # fig.update_layout(xaxis_title='Price', yaxis_title='Notional', title=f'Month {month}')
    # fig.show()

    # This is for the grid subplots

#     fig.add_trace(
#         go.Scatter(
#             x=df_month['Price'],
#             y=df_month['Notional'],
#             mode='markers',
#             name='Actual Data',
#         ),
#         row=row,
#         col=col
#     )

#     fig.add_trace(
#         go.Scatter(
#             x=S_T,
#             y=y,
#             mode='markers',
#             name='Calls spread best fit',            
#         ),
#         row=row,
#         col=col
#     )

# fig.update_layout(height=1000, showlegend=False)
# fig.update_xaxes(title_text="Price", row=4)
# fig.update_yaxes(title_text="Load", col=1)

# fig.show()

# fig.write_image('plot.png', width=1200, height=800)


In [9]:
df_results

,Month,N_L,N_H,K_L,K_H
0,1,1714.156477,2040.005585,29.519546,49.286851
1,2,1531.886324,1822.875044,26.279953,43.331167
2,3,1237.774971,1509.645906,27.20549,41.205118
3,4,981.667541,1239.292492,27.543496,42.75705
4,5,1018.288582,1372.767832,25.078545,42.415879
5,6,1280.782431,1964.814485,23.760542,43.318318
6,7,1547.837729,2454.702817,27.514834,49.851374
7,8,1368.680186,2194.225343,26.189254,46.183684
8,9,1138.711935,1660.196147,27.12094,46.305293
9,10,945.451418,1207.52213,29.287884,47.14269


Pulling forwards, vols (as of COB of 2026-01-14) and comparing the results to obtained from MD-SOS 2025-10 MM

In [10]:
fwd = pull_projection_curves(
    emtdb=emtdb,
    cd='PJM-WESTERN HUB-7x24',
    bp='PJM-BGE-7x24',
    start_dt='2026-01-14',
    end_dt='2026-01-14',
    first_contract_month='202602',
    last_contract_month='202701'
).rename(
    columns={
        'PROJ_LOC_AMT': 'F'
    }
).assign(
    Month=lambda DF: DF.CONTRACT_MONTH % 100
)[['F', 'Month']]

fwd

Function 'execute' executed in 0.1 sec


,F,Month
0,58.876190,2
1,50.093510,3
2,51.296268,4
3,48.100184,5
4,55.983378,6
5,76.691129,7
6,64.150935,8
7,54.727135,9
8,57.182721,10
9,57.971811,11


In [11]:
vols = pull_system_vols(
    emtdb=emtdb,
    cd='PJM-ON',
    eval_dt='2026-01-14',
    first_contract_month='202602',
    last_contract_month='202701'
).assign(
    Month=lambda DF: DF.index % 100
)[['DAILY_VOLATILITY', 'Month']].reset_index(drop=True).rename(
    columns={
        'DAILY_VOLATILITY': 'Sigma'
    }
).drop_duplicates( # Since this pulls all the vols with an eval date after the chosen eval date
    ['Month']
)

vols

Pulling System Vols: commodity=PJM-ON, eval_dt=2026-01-14
Function 'execute' executed in 0.1 sec
Function 'pull_system_vols' executed in 0.1 sec


,Sigma,Month
0,2.170,2
1,0.945,3
2,0.670,4
3,0.670,5
4,0.740,6
5,1.030,7
6,0.940,8
7,0.675,9
8,0.540,10
9,0.535,11


In [12]:
df_results_from_mm = pd.DataFrame([['2/1/2026', 64.31963616], # from MD-SOS 2025-10 MM - priced at BGE with all PVMs of 1 to make it comparable to our results
       ['3/1/2026', 53.47191748],
       ['4/1/2026', 54.61477125],
       ['5/1/2026', 53.51686134],
       ['6/1/2026', 67.3797627],
       ['7/1/2026', 93.26424548],
       ['8/1/2026', 77.90169636],
       ['9/1/2026', 65.25383563],
       ['10/1/2026', 61.48211403],
       ['11/1/2026', 61.52669509],
       ['12/1/2026', 75.07344655],
       ['1/1/2027', 101.548245]],
       columns=['Month', 'Price_from_MM']
       ).assign(
           Month=lambda DF: pd.to_datetime(DF.Month).dt.month
       )

df_results_from_mm

,Month,Price_from_MM
0,2,64.319636
1,3,53.471917
2,4,54.614771
3,5,53.516861
4,6,67.379763
5,7,93.264245
6,8,77.901696
7,9,65.253836
8,10,61.482114
9,11,61.526695


In [13]:
hedges_from_ls = pd.DataFrame(
    [[ 2.        , 24.38095238],
       [ 3.        , 19.19354839],
       [ 4.        , 15.51111111],
       [ 5.        , 16.56989247],
       [ 6.        , 25.45252071],
       [ 7.        , 32.02461339],
       [ 8.        , 28.81911584],
       [ 9.        , 21.69807839],
       [10.        , 15.19354839],
       [11.        , 18.55555556],
       [12.        , 27.79047217],
       [ 1.        , 37.43750749]],
       columns=['Month', 'Hedge_from_LS']
)

hedges_from_ls

,Month,Hedge_from_LS
0,2.0,24.380952
1,3.0,19.193548
2,4.0,15.511111
3,5.0,16.569892
4,6.0,25.452521
5,7.0,32.024613
6,8.0,28.819116
7,9.0,21.698078
8,10.0,15.193548
9,11.0,18.555556


In [16]:
df_results.merge(
    fwd,
    on='Month'
).merge(
    vols,
    on='Month'
).merge(
    df_results_from_mm,
    on='Month'
).merge(
    hedges_from_ls,
    on='Month'
).assign(
    lev=lambda DF: (DF.N_H - DF.N_L) / (DF.K_H - DF.K_L),
    T_mid=lambda DF: (DF.Month - 0.5) / 12, # Time to the middle of the month from today
    F_tilde=lambda DF: DF.F * np.exp(DF.Sigma ** 2 * DF.T_mid),
    CS=lambda DF: DF.apply(
        lambda DF: euro_option_price(DF.F, DF.K_L, DF.T_mid, 0, DF.Sigma) - euro_option_price(DF.F, DF.K_H, DF.T_mid, 0, DF.Sigma),
        axis=1
    ),
    CS_tilde=lambda DF: DF.apply(
        lambda DF: euro_option_price(DF.F_tilde, DF.K_L, DF.T_mid, 0, DF.Sigma) - euro_option_price(DF.F_tilde, DF.K_H, DF.T_mid, 0, DF.Sigma),
        axis=1
    ),
    k_num=lambda DF: DF.F * (DF.N_L + DF.lev * DF.CS_tilde),
    k_denom=lambda DF: DF.N_L + DF.lev * DF.CS,
    k=lambda DF: DF.k_num / DF.k_denom,
    delta_CS_tilde=lambda DF: DF.apply(
        lambda DF: (euro_option_price(DF.F_tilde, DF.K_L, DF.T_mid, 0, DF.Sigma) - delta(DF.F_tilde, DF.K_H, DF.T_mid, 0, DF.Sigma))* np.exp(DF.Sigma ** 2 * DF.T_mid),
        axis=1
    ),
    delta=lambda DF: (DF.k - (DF.N_L + DF.lev * DF.CS_tilde) - (DF.lev * DF.delta_CS_tilde * np.exp(DF.Sigma ** 2 * DF.T_mid))) / -100, # Adjusting for block size to compare to LS, multiplying by -1 to get the hedge volume
    modified_notional=lambda DF: DF.k_denom / 100 # Adjusting for block size
).apply(
    pd.to_numeric
).pipe(
    px.line,
    x='Month',
    # y=['k', 'Price_from_MM'],
    y=['delta', 'Hedge_from_LS', 'modified_notional'],
    # y='Sigma',
    markers=True
) #.write_image('plot.png', width=1200, height=400)